In [1]:
from judo_footage_analysis.utils import get_spark

root = "/cs-share/pradalier/tmp/judo"

spark = get_spark(cores=4, mem="1g")
spark

24/04/11 18:32:43 WARN Utils: Your hostname, gtlpc102.georgiatech-metz.fr resolves to a loopback address: 127.0.1.1; using 192.93.8.102 instead (on interface enp0s31f6)
24/04/11 18:32:43 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
24/04/11 18:32:43 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
24/04/11 18:32:43 WARN SparkConf: Note that spark.local.dir will be overridden by the value set by the cluster manager (via SPARK_LOCAL_DIRS in mesos/standalone/kubernetes and LOCAL_DIRS in YARN).


In [2]:
from pyspark.sql import functions as F

df = (
    spark.read.json(
        f"{root}/models/evaluation_embeddings_logistic_binary/v4/*/*/perf/*.json"
    )
    .withColumn("filename", F.input_file_name())
    .withColumn("feature", F.split(F.col("filename"), "/").getItem(10))
)
df.show()

+--------------------+-----------+-----------+--------------------+-------------------+-------------+---------+-------------------+--------------+----------+------------------+--------------------+--------------------+
|         avg_metrics|      label|metric_name|         std_metrics|        test_metric|test_positive|test_size|          test_time|train_positive|train_size|        train_time|            filename|             feature|
+--------------------+-----------+-----------+--------------------+-------------------+-------------+---------+-------------------+--------------+----------+------------------+--------------------+--------------------+
|[0.7891663451795221]|  is_active|         f1|[0.7891663451795221]|0.37500000000000006|           45|       90|0.47534132993314415|           238|       480|12.309672495000996|file:///cs-share/...|emb_entity_detect...|
|[0.8351040324625366]|is_standing|         f1|[0.8351040324625366]| 0.6602469135802469|            3|       90|0.46770033007

In [3]:
clean = df.select(
    "label",
    "feature",
    F.round(F.col("avg_metrics")[0], 3).alias("avg_train_f1"),
    F.round("test_metric", 3).alias("test_f1"),
    F.col("train_time").cast("integer").alias("train_time"),
).orderBy("label", F.desc("test_f1"))

clean.where(~F.col("feature").contains("lag")).show(truncate=False, n=100)

+-----------+-------------------------------+------------+-------+----------+
|label      |feature                        |avg_train_f1|test_f1|train_time|
+-----------+-------------------------------+------------+-------+----------+
|is_active  |emb_entity_detection_v3_dct_d8 |0.611       |0.783  |7         |
|is_active  |emb_entity_detection_v2_dct_d32|0.81        |0.576  |19        |
|is_active  |emb_entity_detection_v2        |0.843       |0.575  |100       |
|is_active  |emb_vanilla_yolov8n            |0.843       |0.575  |93        |
|is_active  |emb_entity_detection_v2_dctn   |0.844       |0.533  |44        |
|is_active  |emb_entity_detection_v3        |0.817       |0.516  |23        |
|is_active  |emb_entity_detection_v3_dct_d32|0.705       |0.504  |20        |
|is_active  |emb_entity_detection_v3_dct_d16|0.672       |0.483  |14        |
|is_active  |emb_entity_detection_v2_dct_d64|0.817       |0.479  |19        |
|is_active  |emb_entity_detection_v3_dctn   |0.755       |0.475 

In [4]:
clean.where(F.col("feature").contains("dct_d16")).show(truncate=False)

+---------+------------------------------------+------------+-------+----------+
|label    |feature                             |avg_train_f1|test_f1|train_time|
+---------+------------------------------------+------------+-------+----------+
|is_active|emb_entity_detection_v3_dct_d16_lag1|0.686       |0.515  |20        |
|is_active|emb_entity_detection_v3_dct_d16     |0.672       |0.483  |14        |
|is_active|emb_entity_detection_v2_dct_d16_lag1|0.768       |0.408  |19        |
|is_active|emb_entity_detection_v2_dct_d16_lag2|0.767       |0.383  |19        |
|is_active|emb_entity_detection_v2_dct_d16_lag5|0.835       |0.381  |24        |
|is_active|emb_entity_detection_v3_dct_d16_lag2|0.694       |0.381  |21        |
|is_active|emb_entity_detection_v2_dct_d16     |0.789       |0.375  |12        |
|is_active|emb_entity_detection_v2_dct_d16_lag3|0.837       |0.362  |18        |
|is_active|emb_entity_detection_v3_dct_d16_lag5|0.786       |0.345  |16        |
|is_active|emb_entity_detect